<a href="https://colab.research.google.com/github/mkustigian23/Proj2Comp399/blob/main/Comp399Proj2V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import yfinance as yf
import re
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import AutoModelForCausalLM
import torch


def fetch_stock_data(ticker, start_date, end_date):
    """Fetch historical stock data from Yahoo Finance."""
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data


def load_model():
    """Load LLM model of choice."""
    # login(userdata.get('HF_TOKEN2'))
    # model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.3-70B-Instruct", token = userdata.get('HF_TOKEN1'))
    tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-xl")
    model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-xl", token = userdata.get('HF_TOKEN1'))
    return tokenizer, model


def predict_stock_price(tokenizer, model, stock_data, ticker):
    """Use GPT 2 to predict the next closing price based on historical data."""
    input_text = f"Provide the predicted price for tomorrow from {ticker} as a stock price based on the following data: {stock_data.tail(5)}"
    print(stock_data.tail(5))

    inputs = tokenizer(input_text, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs)

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
     # Extract the number using regular expressions
    match = re.search(r'\d+\.?\d*', prediction)

    if match:
        predicted_price = float(match.group(0))
        return predicted_price
    else:
        return "No numerical prediction found."
    return prediction


def main():
    ticker = "AAPL"  # Change to any stock symbol, for the example here I used Apple as it is one of the most well known stocks.
    start_date = "2025-03-01"
    end_date = "2025-03-26"

    print(f"Fetching stock data for {ticker}...")
    stock_data = fetch_stock_data(ticker, start_date, end_date)

    print("Loading LLM model...")
    tokenizer, model = load_model()

    print("Predicting stock price...")
    prediction = predict_stock_price(tokenizer, model, stock_data, ticker)

    print(f"Predicted next closing price for {ticker}: {prediction}")


if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed

Fetching stock data for AAPL...
Loading LLM model...


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Predicting stock price...
Price            Close        High         Low        Open    Volume
Ticker            AAPL        AAPL        AAPL        AAPL      AAPL
Date                                                                
2025-03-19  215.240005  218.759995  213.750000  214.220001  54385400
2025-03-20  214.100006  217.490005  212.220001  213.990005  48862900
2025-03-21  218.270004  218.839996  211.279999  211.559998  94127800
2025-03-24  220.729996  221.479996  218.580002  221.000000  44299500
2025-03-25  223.750000  224.100006  220.080002  220.770004  34493600
Predicted next closing price for AAPL: 2025.0
